
# Handout 13

## **O que são Sinais?**

Em sistemas operacionais compatíveis com POSIX, um **sinal** é uma notificação assíncrona enviada a um processo para informá-lo sobre a ocorrência de um evento específico. Esses eventos podem incluir exceções de hardware, condições de sistema ou ações do usuário. Os sinais permitem que processos reajam a eventos inesperados ou que se comuniquem entre si de maneira eficiente.

## **Funcionamento dos Sinais**

Quando um sinal é enviado a um processo, o sistema operacional interrompe temporariamente a execução normal desse processo para lidar com o sinal. O processo pode definir uma função específica para tratar o sinal (chamada de *signal handler*) ou adotar o comportamento padrão associado a esse sinal.  
**Exemplo**: O sinal `SIGINT`, geralmente gerado quando o usuário pressiona `Ctrl+C` no terminal, tem como ação padrão a terminação do processo.

## **Principais Sinais POSIX**

Alguns dos sinais mais comuns definidos pelo padrão POSIX incluem:

- **SIGINT (2)**: Interrupção do terminal (geralmente gerado por `Ctrl+C`).
- **SIGKILL (9)**: Finalização imediata do processo (não pode ser capturado ou ignorado).
- **SIGTERM (15)**: Solicitação de terminação do processo.
- **SIGSEGV (11)**: Violação de segmentação (acesso inválido à memória).
- **SIGCHLD (17)**: Indica que um processo filho terminou ou foi interrompido.

Cada sinal possui um número associado e uma ação padrão que o sistema operacional executa caso o processo não defina um tratador específico.


## **Enviando Sinais**

Sinais podem ser enviados de diversas maneiras:

1. **A partir do terminal**: Certas combinações de teclas enviam sinais para o processo em execução no terminal.  
   **Exemplo**: `Ctrl+C` envia `SIGINT`, enquanto `Ctrl+Z` envia `SIGTSTP` (suspende o processo).

2. **Usando a chamada de sistema `kill`**: Um processo pode enviar sinais a outro utilizando a função `kill`:

    ```c
    #include <signal.h>
    #include <unistd.h>

    int kill(pid_t pid, int sig);
    ```

    Onde `pid` é o identificador do processo de destino e `sig` é o sinal a ser enviado.  
    **Exemplo**: Para enviar `SIGTERM` a um processo com PID `1234`:

    ```c
    kill(1234, SIGTERM);
    ```

3. **Através do comando `kill` no shell**:  
   O comando `kill` permite que usuários enviem sinais a processos a partir da linha de comando:

    ```bash
    kill -SIGTERM 1234
    ```

    Ou, utilizando o número do sinal:

    ```bash
    kill -15 1234
    ```

---

## **Capturando Sinais**

Processos podem definir funções específicas para tratar sinais, permitindo uma resposta personalizada a eventos.  
Isso é feito utilizando a função `signal` ou `sigaction`. A função `signal` possui a seguinte assinatura:

```c
#include <signal.h>

void (*signal(int sig, void (*func)(int)))(int);

**Exemplo:** Para capturar o sinal SIGINT e definir uma função tratadora:

In [ ]:
#include <stdio.h>
#include <signal.h>

void handle_sigint(int sig) {
    printf("Recebi o sinal SIGINT (%d)\n", sig);
}

int main() {
    signal(SIGINT, handle_sigint);
    while (1) {
        // Loop infinito
    }
    return 0;
}


## **Exemplo Prático: Envio e Captura de Sinais**

Vamos criar dois programas: um que envia um sinal e outro que o captura.

**Programa que captura o sinal (captura_sinal.c)**

In [ ]:
#include <stdio.h>
#include <signal.h>
#include <unistd.h>

void handle_sigusr1(int sig) {
    printf("Recebi o sinal SIGUSR1 (%d)\n", sig);
}

int main() {
    signal(SIGUSR1, handle_sigusr1);
    printf("PID do processo: %d\n", getpid());
    while (1) {
        pause(); // Aguarda por sinais
    }
    return 0;
}


**Programa que envia o sinal (envia_sinal.c)**

In [ ]:
#include <stdio.h>
#include <signal.h>
#include <stdlib.h>

int main(int argc, char *argv[]) {
    if (argc != 2) {
        fprintf(stderr, "Uso: %s <PID>\n", argv[0]);
        exit(EXIT_FAILURE);
    }
    pid_t pid = atoi(argv[1]);
    if (kill(pid, SIGUSR1) == -1) {
        perror("Erro ao enviar sinal");
        exit(EXIT_FAILURE);
    }
    printf("Sinal SIGUSR1 enviado para o processo %d\n", pid);
    return 0;
}


![execução](image.png)
